### Importing libraries

In [ ]:
# Import for exploration and visualization
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import os

# import sklearn and xgboost libraries
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.model_selection import StratifiedKFold
import joblib


### Data exploration ###

In [ ]:
# Use panda to read the csv, using the head() method you can check the first 5 rows
train = pd.read_csv("../data/raw/train.csv")
test = pd.read_csv("../data/raw/test.csv")
train.head()

In [ ]:
test.head()

In [ ]:
train.info()
# The .info() method gives a quick summary of the df structure.
# In this case, the dataset has 891 rows and 12 columns.
# For each column, it shows:
#   - The column index (position in the df)
#   - The column name
#   - The number of non-null (non-missing) values
#   - The data type (dtype) of the column
# This information is useful for:
#   - Detecting missing values
#   - Understanding data types before cleaning or transformation
#   - Getting a sense of the dataset size and memory usage

In [ ]:
train.describe() # describe gives you statistical summaries of the df
# count	Number of non-missing values
# mean  -   Average value
# std   -   Standard deviation (spread of data)
# min   -	Minimum value
# 25%   -	1st quartile (25% of data ≤ this value)
# 50%   -   Median (middle value)
# 75%   -	3rd quartile (75% of data ≤ this value)
# max   -	Maximum value

In [ ]:
train.describe(include='object')
# you can use this to include objects to get info in the non-numeric columns
# unique    -   number of distinct values
# top       -   most common value
# freq      -   how many times that top value appears   

In [ ]:
# Using describe() shows that most passengers did not survive. 
# This means the 'Survived' column can be used as a target variable to analyze and compare the characteristics of survivors vs. non-survivors in the dataset.
survi_mean = train.groupby("Survived").mean(numeric_only=True)
survi_0 = survi_mean.iloc[0,:] # avergage values of passenger that DID not survive
survi_1 = survi_mean.iloc[1,:] # avergage values of passenger DID not survive
# Relative difference calculation
# This highlights which features differ most proportionally between the two groups, helping identify potential predictors of survival.
abs((survi_1 - survi_0) / (survi_0 + survi_1))

In [ ]:
# The relative differences show that Fare, Parch, and Pclass matter the most for survival.
# Let's explore these features further to understand their impact,
# which will help us gauge how much importance our XGBoost model should assign to them.
# Fare by survival
sns.violinplot(x="Fare", hue="Survived", data=train)
plt.title("Passenger Fare by Survival")
plt.show

In [ ]:
# Pclass by survival
sns.countplot(x="Pclass", hue="Survived", data=train)
plt.title("Passenger Class by Survival")
plt.show()

In [ ]:
# Parch by survival
sns.violinplot(x="Parch", hue="Survived", data=train)
plt.title("Passenger Parch by Survival")
plt.show

In [ ]:
# For context, let's see the PassengerId value impact so we can see how little it matters
plt.figure(figsize=(10,4))
plt.scatter(train["PassengerId"], train["Survived"], alpha=0.3)
plt.xlabel("PassengerId")
plt.ylabel("Survived")
plt.title("PassengerId vs Survived")
plt.yticks([0,1])
plt.show

In [ ]:
# Calculate the correlation matrix for all numeric features in the dataset.
# Then plot a heatmap to visually inspect the strength and direction of linear relationships between pairs of numeric variables.
# The heatmap's colors and annotated values help identify which features are strongly
# positively or negatively correlated, which is useful for feature selection and understanding data structure.
corr = train.select_dtypes(include="number").corr()
plt.subplots(figsize=(12, 12))
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, annot=True)
plt.show()

In [ ]:
# Analyzing this, we can check that, for example, the correlation with Survival is:
# -0.005 with PassengerID, Almost no correlation, meaning that it has practically no correlation
# 1 with Survived, perfect correlation with itself, for obvious reasons
# -0.34 with Pclass, Moderate negative correlation, higher class (lower number) means MORE likely to survive
# -0.077 with Age, Weak negative correlation, younger passengers slightly more likely to survive
# -0.035 with SibSp, Weak negative correlation, having siblings/spouses means almost nothing when it comes to survive
# 0.082 with Parch, Weak positive correlation, having parents/childrens means almost nothing when it comes to survive
# 0.26 with Fare, Moderate positive correlation, paying higher fares increases chance of survival+

### Preprocessing data ###

In [ ]:
print(train["Name"].head())

In [ ]:
# Extract title using regex
train["title"] = train["Name"].str.extract(r",\s*([^\.\"]+)\.", expand=False)

# Map titles to numeric codes using a dictionary
title_mapping = {
    "Mr": 1,
    "Master": 3,
    "Ms": 4, "Mlle": 4, "Miss": 4,
    "Mme": 5, "Mrs": 5
}

# Map titles, assign 2 to all others (including unknown)
train["title"] = train["title"].map(title_mapping).fillna(2).astype(int)

print(train["title"])


In [ ]:
# Convert Fare to True if above average, else False
meanfare = train["Fare"].mean()
train["Fare"] = train["Fare"] > meanfare
print(train["Fare"])

In [ ]:
# With that, lets start from droping some values we don't need.
train = train.drop(["PassengerId","Name","Ticket"], axis="columns")
train.info()

In [ ]:
# Lets replace the values of sex with numbers so its easier for the model to work with
train["Sex"] = train["Sex"].replace(["male", "female"], [0,1])
train.info()

In [ ]:
# Since cabin either is null or has a random alphanumeric value, lets make it a boolean
train["Cabin"] = train["Cabin"].isna()
train.info()

In [ ]:
mean_ages = train.groupby("title")["Age"].agg("mean")
print(mean_ages)

In [ ]:
# Mapping from title to age to fill missing values
title_age_map = {
    1:32,
    2:45,
    3:7,
    4:23,
    5:35,
}

# Fill missing Age values based on title
train["Age"] = train["Age"].fillna(train["title"].map(title_age_map))

# Fill any remaining missings using the mean
mean_age =  train["Age"].mean()
train["Age"] = train["Age"].fillna(mean_age)
train.head()

In [ ]:
# Converts categorical variables in the DataFrame into one-hot encoded columns, creating new binary columns for each category value,so machine learning models can process categorical data easily
train = pd.get_dummies(train)
# So now we do this again after the data treatment
survi_mean = train.groupby("Survived").mean()
survi_0 = survi_mean.iloc[0,:] 
survi_1 = survi_mean.iloc[1,:] 
abs((survi_1 - survi_0) / (survi_0 + survi_1))

### Modeling ###

In [ ]:
# Remove the Survived column from train and assign it to y (target variable), leaving train with only feature columns which we assign to x (input features)
y = train["Survived"]
x = train.drop(columns=["Survived"])


In [ ]:
# Basic XGBoost classifier usage
model = XGBClassifier()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)
model.fit(x_train, y_train)
print(model.score(x_test, y_test))

In [ ]:
# We check the prediction the model does for survivors
pred = model.predict(x_test)
print("Survived", sum(pred != 0))

In [ ]:
# We check the prediction the model does for non-survivors
pred = model.predict(x_test)
print("Not Survived", sum(pred == 0))

In [ ]:
# We use a confusion matrix to check how many TN, FP, FN and TP we have.
# [[TN, FP],   <-- First row: True Negatives (TN), False Positives (FP)
#  [FN, TP]]   <-- Second row: False Negatives (FN), True Positives (TP)
conmatrix = confusion_matrix(y_test, pred)
conmatrix

In [ ]:
# We obtained 86% accuracy with XGBoost model, now we might want to adjust its hyperparameter using GridSearch
cv_splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

model = XGBClassifier(random_state=42)  # fixes XGBoost's own RNG
ran_grid = {"eta": np.linspace(0, 0.5, num=12)}

ran = GridSearchCV(model, ran_grid, cv=cv_splitter)
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=11
)

ran.fit(x_train, y_train)
print(ran.best_score_)
print(ran.best_params_)

In [ ]:
# We adjust max_depth
model = XGBClassifier(eta = 0.045454545454545456)
random_grid = {"max_depth": range(1,20,1)}

ran = GridSearchCV(model, random_grid, cv=5)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=11)
ran.fit(x_train, y_train)
print(ran.best_score_)
ran.best_params_

In [ ]:
# We adjust min_child_weight
model = XGBClassifier(eta = 0.045454545454545456, max_depth = 6)
random_grid = {"min_child_weight": range(1,20,1)}

ran = GridSearchCV(model, random_grid, cv=5)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=11)
ran.fit(x_train, y_train)
print(ran.best_score_)
ran.best_params_


In [ ]:
# We adjust gamma
model = XGBClassifier(eta = 0.045454545454545456, max_depth = 6, min_child_weight=1)
random_grid = {"gamma" : [i / 10.0 for i in range(0, 11)]}

ran = GridSearchCV(model, random_grid, cv=5)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=11)
ran.fit(x_train, y_train)
print(ran.best_score_)
ran.best_params_

In [ ]:
# We adjust subsample
model = XGBClassifier(eta=0.045454545454545456, max_depth=6, min_child_weight=1, gamma=0.3)
random_grid = {"subsample": [i / 100.0 for i in range(50, 101, 5)]}

ran = GridSearchCV(model, random_grid, cv=5)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=11)
ran.fit(x_train, y_train)
print(ran.best_score_)
print(ran.best_params_)

In [ ]:
# We adjust comsample_bytree
model = XGBClassifier(eta=0.045454545454545456, max_depth=6, min_child_weight=1, gamma=0.3, subsample=1.0)
random_grid = {"colsample_bytree": [i / 10.0 for i in range(5, 11)]}  # 0.5 to 1.0

ran = GridSearchCV(model, random_grid, cv=5)
ran.fit(x_train, y_train)
print(ran.best_score_)
print(ran.best_params_)

In [ ]:
# We adjust reg_alpha
model = XGBClassifier(
    eta=0.045454545454545456,
    max_depth=6,
    min_child_weight=1,
    gamma=0.3,
    subsample=1.0,
    colsample_bytree=1.0
)
random_grid = {"reg_alpha": [0, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10]}

ran = GridSearchCV(model, random_grid, cv=5)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=11)
ran.fit(x_train, y_train)
print(ran.best_score_)
print(ran.best_params_)

In [ ]:
# We check how we doing isn 92,17% accuracy
model = XGBClassifier(
    eta=0.045454545454545456,
    max_depth=6,
    min_child_weight=1,
    gamma=0.3,
    subsample=1.0,
    colsample_bytree=1.0,
    reg_alpha = 0.05
)

model.fit(x, y)
print(model.score(x_test, y_test))
pred = model.predict(x_test)
print("Survived", sum(pred!=0))
print("Not Survived", sum(pred==0))

conmatrix = confusion_matrix(y_test, pred)
conmatrix

We trained the XGBoost model with the best hyperparameters found via GridSearchCV
We achieve 92.17% accuracy on the test set (x_test, y_test)
Then we predict the 'Survived' labels and print how many passengers the model predicts survived and not survived
Finally, we compute the confusion matrix to evaluate performance in detail:
 - 113 true negatives (correctly predicted not survived)
 - 52 true positives (correctly predicted survived)
 - 5 false positives (predicted survived but actually did not survive)
 - 9 false negatives (predicted not survived but actually survived)

In [ ]:
ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

model_path = os.path.join(ROOT, "models", "titanic_xgb_92_18.pkl")
data_path = os.path.join(ROOT, "data", "processed_titanic.csv")

joblib.dump(model, model_path)
# Save the fully processed dataset for testing or deployment
train.to_csv(data_path, index=False)

# print("Model saved to:", model_path)
# print("Data saved to:", data_path)